# Gold Layer - Data Modeling

### The Medallion architecture 
----

  ![schema](images/medallion_architecture.png)

 #### Star Schema 
 ----
 
  ![schema](images/star_schema.png)

In [5]:
from IPython.display import display,HTML
display(HTML("<style>pre {white-space:pre !important;}</style> "))

In [303]:
from sqlalchemy import create_engine,ForeignKey,Column,String,Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine,ForeignKey,Column,String,Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,relationship
from datetime import  datetime
from pyspark.sql.functions import lit
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# to extract keywords
nltk.download('punkt')
nltk.download('stopwords')

In [304]:
from sqlalchemy import create_engine
from sqlalchemy.exc import ProgrammingError

### Spark Data frame To DW  using SQLAlchemy (ORM pattern)

In [65]:
findspark.init()

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

In [66]:
patents = spark.read.json('transformed-data/transformed_data.json')

In [355]:
patents.show(4)

+--------------------+--------------------+----------------+---------------------+------------------+-----------------------+------------------------+--------------+-------+----------------------+-----------------------+----------+--------------------+--------------------+-------------+----------+------+--------------------+
|       abstract_text|          applicants|application_date|assignee_name_current|assignee_name_orig|backward_cite_no_family|backward_cite_yes_family|          code|country|forward_cite_no_family|forward_cite_yes_family|grant_date|       inventor_name|                link|priority_date|  pub_date|source|               title|
+--------------------+--------------------+----------------+---------------------+------------------+-----------------------+------------------------+--------------+-------+----------------------+-----------------------+----------+--------------------+--------------------+-------------+----------+------+--------------------+
|The present discl.

In [75]:
patents.printSchema()

root
 |-- abstract_text: string (nullable = true)
 |-- applicants: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- application_date: string (nullable = true)
 |-- assignee_name_current: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- assignee_name_orig: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- backward_cite_no_family: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- patent_number: string (nullable = true)
 |    |    |-- priority_date: string (nullable = true)
 |    |    |-- pub_date: string (nullable = true)
 |-- backward_cite_yes_family: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- patent_number: string (nullable = true)
 |    |    |-- priority_date: string (nullable = true)
 |    |    |-- pub_date: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- forward_cit

In [ ]:
def extract_keywords(text, n=3):
    tokens = word_tokenize(text)
    
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    
    fdist = nltk.FreqDist(filtered_tokens)
    
    keywords = [word for word, _ in fdist.most_common(n)]  # Extract only the words without frequencies
    
    return keywords
    

In [53]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from datetime import datetime

In [ ]:
Base=declarative_base()

class Patent(Base):
    __tablename__ ="DimPatent"
    code_patent=Column("code_patent",String,primary_key=True)
    source = Column("source",String)
    link = Column("link",String)
    abstract = Column("abstract",String)
    id_title = Column("id_title",Integer,ForeignKey("DimTitle.id_title"))
    id_country = Column("id_country",Integer,ForeignKey("DimCountry.id_country"))

    title = relationship('Title',back_populates='patent')
    country = relationship('Country',back_populates='patents')  

    fact_publication = relationship('FactPublication',uselist=False,back_populates='patent')
    fact_grant = relationship('FactGrant',uselist=False,back_populates='patent')
    fact_application = relationship('FactApplication',uselist=False,back_populates='patent')
    fact_priority = relationship('FactPriority',uselist=False,back_populates='patent')





class Title(Base):
    __tablename__ ="DimTitle"
    id_title=Column("id_title",Integer,primary_key=True,autoincrement=True)
    title = Column("title",String)

    patent = relationship('Patent',back_populates='title')
    fact_keyword = relationship('FactKeyword',back_populates='title')


     

class Country(Base):
    __tablename__ ="DimCountry"
    id_country=Column("id_country",Integer,primary_key=True,autoincrement=True)
    country_name = Column("country_name",String)

    patents = relationship('Patent',back_populates='country')
    fact_keyword = relationship('FactKeyword',back_populates='country')



class Inventor(Base):
    __tablename__ ="DimInventor"
    id_inventor=Column("id_inventor",Integer,primary_key=True,autoincrement=True)
    inventor_full_name = Column("inventor_full_name",String)

    fact_publication = relationship('FactPublication',back_populates='inventor')
    fact_application = relationship('FactApplication',back_populates='inventor')
    fact_grant = relationship('FactGrant',back_populates='inventor')
    fact_priority = relationship('FactPriority',back_populates='inventor')
    fact_keyword = relationship('FactKeyword',back_populates='inventor')

class Assignee(Base):
    __tablename__ ="DimAssignee"
    id_assignee=Column("id_assignee",Integer,primary_key=True,autoincrement=True)
    assignee_name = Column("assignee_name",String)
    type = Column("type",String)

    fact_grant = relationship('FactGrant',back_populates='assignee')
    fact_keyword = relationship('FactKeyword',back_populates='assignee')

class Applicant(Base):
    __tablename__ ="DimApplicant"
    id_applicant=Column("id_applicant",Integer,primary_key=True,autoincrement=True)
    applicant_name = Column("applicant_name",String)

    fact_application = relationship('FactApplication',back_populates='applicant')

class Keyword(Base):
    __tablename__ = "DimKeyword"
    id_keyword = Column("id_keyword", Integer, primary_key=True, autoincrement=True)
    keyword = Column("keyword", String)

    fact_keyword = relationship('FactKeyword',back_populates='keyword')

class Time(Base):
    __tablename__ = "DimTime"

    id_time = Column("id_time",Integer,primary_key=True, autoincrement=True)
    year = Column("year", Integer)
    month = Column("month", Integer)
    day =  Column("day", Integer)

    fact_publication = relationship('FactPublication',back_populates='time')
    fact_keyword = relationship('FactKeyword',back_populates='time')
    fact_application = relationship('FactApplication',back_populates='time')
    fact_priority = relationship('FactPriority',back_populates='time')
    fact_grant = relationship('FactGrant',back_populates='time')

    def __init__(self,date):
        if date is None:  
            self.year = 0
            self.month = 0
            self.day = 0 
        else :    
            date_obj = datetime.strptime(date,'%Y-%m-%d')
            self.year = date_obj.year
            self.month = date_obj.month
            self.day = date_obj.day


class FactPublication(Base):
    __tablename__ = "FactPublication"

    id_inventor = Column("id_inventor", Integer, ForeignKey("DimInventor.id_inventor"), primary_key=True)
    id_time = Column("id_time", Integer, ForeignKey("DimTime.id_time"), primary_key=True)
    code_patent = Column("code_patent", String, ForeignKey("DimPatent.code_patent"), primary_key=True)

    inventor = relationship('Inventor',back_populates='fact_publication')
    time = relationship('Time',back_populates='fact_publication')
    patent = relationship('Patent',uselist=False,back_populates='fact_publication')

class FactGrant(Base):
    __tablename__ = "FactGrant"
    
    id_fact_grant = Column("id_fact_grant",Integer,primary_key=True, autoincrement=True)

    id_assignee = Column("id_assignee", Integer, ForeignKey("DimAssignee.id_assignee"))
    id_inventor = Column("id_inventor", Integer, ForeignKey("DimInventor.id_inventor"))
    id_time = Column("id_time", Integer, ForeignKey("DimTime.id_time"))
    code_patent = Column("code_patent", fString, ForeignKey("DimPatent.code_patent"))

    inventor = relationship('Inventor',back_populates='fact_grant')
    time = relationship('Time',back_populates='fact_grant')
    patent = relationship('Patent',uselist=False,back_populates='fact_grant')
    assignee = relationship('Assignee',back_populates='fact_grant')

class FactApplication(Base):
    __tablename__ = "FactApplication"

    id_applicant = Column("id_applicant", Integer, ForeignKey("DimApplicant.id_applicant"), primary_key=True)
    id_inventor = Column("id_inventor", Integer, ForeignKey("DimInventor.id_inventor"), primary_key=True)
    id_time = Column("id_time", Integer, ForeignKey("DimTime.id_time"), primary_key=True)
    code_patent = Column("code_patent", String, ForeignKey("DimPatent.code_patent"), primary_key=True)

    inventor = relationship('Inventor',back_populates='fact_application')
    time = relationship('Time',back_populates='fact_application')
    patent = relationship('Patent',uselist=False,back_populates='fact_application')
    applicant = relationship('Applicant',back_populates='fact_application')

class FactPriority(Base):
    __tablename__ = "FactPriority"

    id_inventor = Column("id_inventor", Integer, ForeignKey("DimInventor.id_inventor"), primary_key=True)
    id_time = Column("id_time", Integer, ForeignKey("DimTime.id_time"), primary_key=True)
    code_patent = Column("code_patent", String, ForeignKey("DimPatent.code_patent"), primary_key=True)

    inventor = relationship('Inventor',back_populates='fact_priority')
    time = relationship('Time',back_populates='fact_priority')
    patent = relationship('Patent',uselist=False,back_populates='fact_priority')

class FactKeyword(Base):
    __tablename__ = "FactKeyword"

    id_fact_keyword = Column("id_fact_keyword",Integer,primary_key=True, autoincrement=True)

    id_title = Column("id_title", Integer, ForeignKey("DimTitle.id_title"))
    id_inventor = Column("id_inventor", Integer, ForeignKey("DimInventor.id_inventor"))
    id_country = Column("id_country", Integer, ForeignKey("DimCountry.id_country"))
    id_time = Column("id_time", Integer, ForeignKey("DimTime.id_time"))
    id_assignee = Column("id_assignee", Integer, ForeignKey("DimAssignee.id_assignee"))
    id_keyword = Column("id_keyword", Integer, ForeignKey("DimKeyword.id_keyword"))

    inventor = relationship('Inventor',back_populates='fact_keyword')
    time = relationship('Time',back_populates='fact_keyword')
    assignee = relationship('Assignee',back_populates='fact_keyword')
    keyword = relationship('Keyword',back_populates='fact_keyword')
    title = relationship('Title',back_populates='fact_keyword')
    country = relationship('Country',back_populates='fact_keyword')


In [4]:
username = 'postgres'
password = 'aymane2002'
server_name = 'localhost'
port = 5432
db_name = 'Patent'

In [ ]:
engine = create_engine(f'postgresql://{username}:{password}@{server_name}:{port}/{db_name}',echo=True)

Base.metadata.create_all(bind=engine)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
def insert_row(row):
    global session
    try:


        title = session.query(Title).filter_by(title=row['title']).first()
        if title is None:
            title = Title(title= row['title'])
            session.add(title)
            session.commit()

        
        country = session.query(Country).filter_by(country_name=row['country']).first()
        if country is None:
            country = Country(country_name=row['country'])
            session.add(country)
            session.commit()
            
            

            
            
        patent = session.query(Patent).filter_by(code_patent=row['code']).first()
        if patent is None:
            patent = Patent(
                code_patent=row['code'],
                source=row['source'],
                link=row['link'],
                abstract=row['abstract_text'],
                title=title,
                country=country)
            session.add(patent)
            session.commit()
        
        session.commit()
        
        pub_time = Time(date=row['pub_date'])
        temp_pub_time = session.query(Time).filter_by(year=pub_time.year,month=pub_time.month,day=pub_time.day).first()

        if temp_pub_time is not None:
            pub_time = temp_pub_time
        session.add(pub_time)

            
        app_time = Time(date=row['application_date'])
        temp_app_time = session.query(Time).filter_by(year=app_time.year,month=app_time.month,day=app_time.day).first()

        if temp_app_time is not None:
            app_time = temp_app_time
        session.add(app_time)

        
        grant_time = Time(date=row['grant_date'])
        temp_grant_time = session.query(Time).filter_by(year=grant_time.year,month=grant_time.month,day=grant_time.day).first()

        if temp_grant_time is not None:
            grant_time = temp_grant_time
        session.add(grant_time)

            
        prio_time = Time(date=row['priority_date'])
        temp_priority_time = session.query(Time).filter_by(year=prio_time.year,month=prio_time.month,day=prio_time.day).first()

        if temp_priority_time is not None:
            priority_time = temp_priority_time
        session.add(prio_time)


        
        assignee_names_orig = row['assignee_name_orig'] 
        assignee_names_cur = row['assignee_name_current']
        inventor_names = row['inventor_name']
        applicant_names = row['applicants']
        keywords_names=extract_keywords(row['title'])

        
        keywords = []
        inventors = []
        applicants = []
        assignees_names_orig = []
        assignees_names_cur = []
        
        
        
        for name in inventor_names:
            inventor = session.query(Inventor).filter_by(inventor_full_name=name).first()
            if inventor is None:
                inventor=Inventor(inventor_full_name=name)
                session.add(inventor)
                inventors.append(inventor)
            
        
            
        for name in applicant_names:
            applicant = session.query(Applicant).filter_by(applicant_name=name).first()
            if applicant is None:
                applicant=Applicant(applicant_name=name)
                session.add(applicant)
                applicants.append(applicant)
        
        for name in assignee_names_orig:
            assignee = session.query(Assignee).filter_by(assignee_name=name,type="origin").first()
            if assignee is None:
                assignee=Assignee(assignee_name=name,type="origin")
                session.add(assignee)
            assignees_names_orig.append(assignee)

        for name in assignee_names_cur:
            assignee = session.query(Assignee).filter_by(assignee_name=name,type="current").first()
            if assignee is None:
                assignee=Assignee(assignee_name=name,type="current")
                session.add(assignee)
            assignees_names_cur.append(assignee)
        
        for keyword in keywords_names:
            key = session.query(Keyword).filter_by(keyword=keyword).first()
            if key is None:
                key=Keyword(keyword=keyword)
                session.add(key)
            keywords.append(key)

        session.commit()
        
        fact_publications = [FactPublication(inventor=inventor, time=pub_time, patent=patent) for inventor in inventors]
        fact_priorities = [FactPriority(inventor=inventor,time=prio_time,patent=patent) for inventor in inventors]
        fact_applications = []
        fact_grants = []
        fact_keywords = []
    
    
        
        for inventor in inventors:
            for applicant in applicants:
                fact_applications.append(FactApplication(inventor=inventor,time=app_time,patent=patent,applicant=applicant))
       
        
        for inventor in inventors:
            for origin in assignees_names_orig:
                fact_grants.append(FactGrant(inventor=inventor,time=grant_time,patent=patent,assignee=origin))
            for current in assignees_names_cur:
                fact_grants.append(FactGrant(inventor=inventor,time=grant_time,patent=patent,assignee=current))
        
        for inventor in inventors:
            for keyword in keywords:
                for origin in assignees_names_orig:               
                    fact_keywords.append(FactKeyword(keyword=keyword, title=title,inventor=inventor,time=prio_time,assignee=origin,country=country))
                for current in assignees_names_cur:
                    fact_keywords.append(FactKeyword(keyword=keyword, title=title,inventor=inventor,time=prio_time,assignee=current,country=country))

       
        session.add_all(fact_publications+fact_applications+fact_grants+fact_priorities+fact_keywords)
        session.commit()

        
        print("++++++++++++++++++ done +++++++++++++++++")


    except Exception as e:
        session.rollback()
        raise e

    finally:
        session.close()

for row in patents.collect():
    insert_row(row)

session.close()


## next step : data analysis

#### Analyse  1D
- total_patent
- total_inventor
- total_assignee_curr
- total_assignee_orig
- total_country
- total_keyword
- total_organisation

### Analyse  2D 
- inventor with inventor
- country with country
- keyword with keyword
- keyword with country
- keyword with inventor
- patent with time
- country with time
- inventor with time
- organisation with time
- organisation with country
- organisation with inventor
- organisation with keyword

### Analyse  3D 
- inventor with inventor - time 
- country with country - time
- keyword with keyword - time
- keyword with country - time
- keyword with inventor - time
- organisation - country - time
- organisation - inventor - time
- organisation - keyword - time